## Data wrangling and cleaning

This notebook shows how I loaded, wrangled, and cleaned two datasets Life Expectancy and per Capita GDP. In order to rank countries by those metrics for the year 2022, and see how those ranks match using Kendall's tau a measure or rank correlation. The datasets can be found in Data/RAW

The cleaned--final datasets can be found in Data/CLEAN and this script shows how to get from the raw to the cleaned data. 

### Steps 1 + 2 Import libraries and specify paths to input data and path for cleaned data

In [1]:
## STEP 1 ------Import libraries--------------------
# imports 
import pandas as pd
import numpy as np
from pathlib import Path

## STEP 2 ----------specify paths to input data and path for cleaned data---------------
# Define base path as one level up from the current notebook directory
base_dir = Path.cwd().parent  # assumes notebook is in DataWranglingScripts/

raw_dir = base_dir / "RAW"
clean_dir = base_dir / "CLEAN"

gdp_path = raw_dir / "API_NY.GDP.PCAP.CD_DS2_en_csv_v2_134819.csv"
lifeexp_path = raw_dir / "API_SP.DYN.LE00.MA.IN_DS2_en_csv_v2_126205.csv"

print("Base directory:", base_dir)
print("GDP path:", gdp_path)
print("Life Expectancy path:", lifeexp_path)


Base directory: c:\Users\rl587\PROJECTS\NAULearningDataScience\Data
GDP path: c:\Users\rl587\PROJECTS\NAULearningDataScience\Data\RAW\API_NY.GDP.PCAP.CD_DS2_en_csv_v2_134819.csv
Life Expectancy path: c:\Users\rl587\PROJECTS\NAULearningDataScience\Data\RAW\API_SP.DYN.LE00.MA.IN_DS2_en_csv_v2_126205.csv


### Step 3 Load and Inspect Raw Data


In [2]:
# Load CSVs (skip metadata rows)
gdp = pd.read_csv(gdp_path, skiprows=4)
life = pd.read_csv(lifeexp_path, skiprows=4)

#print("GDP columns:", gdp.columns[:10].tolist())
#print("Life Expectancy columns:", life.columns[:10].tolist())

gdp.head()

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2016,2017,2018,2019,2020,2021,2022,2023,2024,Unnamed: 69
0,Aruba,ABW,GDP per capita (current US$),NY.GDP.PCAP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,27441.529662,28440.051964,30082.127645,31096.205074,22855.932320,27200.061079,30559.533535,33984.790620,NaN,NaN
1,Africa Eastern and Southern,AFE,GDP per capita (current US$),NY.GDP.PCAP.CD,186.121835,186.941781,197.402402,225.440494,208.999748,226.876513,...,1329.807285,1520.212231,1538.901679,1493.817938,1344.103210,1522.393346,1628.318944,1510.742951,1567.635839,NaN
2,Afghanistan,AFG,GDP per capita (current US$),NY.GDP.PCAP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,522.082216,525.469771,491.337221,496.602504,510.787063,356.496214,357.261153,413.757895,NaN,NaN
3,Africa Western and Central,AFW,GDP per capita (current US$),NY.GDP.PCAP.CD,121.939925,127.454189,133.827044,139.008291,148.549379,155.565216,...,1630.039447,1574.230560,1720.140280,1798.340685,1680.039332,1765.954788,1796.668633,1599.392983,1284.154441,NaN
4,Angola,AGO,GDP per capita (current US$),NY.GDP.PCAP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,1807.952941,2437.259712,2538.591391,2189.855714,1449.922867,1925.874661,2929.694455,2309.534130,2122.083690,NaN


In [3]:
life.head()

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2016,2017,2018,2019,2020,2021,2022,2023,2024,Unnamed: 69
0,Aruba,ABW,"Life expectancy at birth, male (years)",SP.DYN.LE00.MA.IN,60.746000,61.114000,61.371000,61.601000,61.824000,62.047000,...,72.691000,72.802000,72.880000,73.047000,72.344000,70.794000,73.537000,73.702000,NaN,NaN
1,Africa Eastern and Southern,AFE,"Life expectancy at birth, male (years)",SP.DYN.LE00.MA.IN,42.448909,42.681453,43.117691,43.341702,43.682342,43.891466,...,59.732220,60.106818,60.796394,61.289264,61.048093,60.351874,61.765707,62.395547,NaN,NaN
2,Afghanistan,AFG,"Life expectancy at birth, male (years)",SP.DYN.LE00.MA.IN,32.136000,32.626000,33.098000,33.543000,34.004000,34.438000,...,60.661000,59.388000,59.175000,59.910000,58.455000,57.052000,63.941000,64.467000,NaN,NaN
3,Africa Western and Central,AFW,"Life expectancy at birth, male (years)",SP.DYN.LE00.MA.IN,35.925673,36.181219,37.031000,37.267402,37.504040,37.778747,...,55.467065,55.660707,56.066077,56.145037,56.219168,56.296796,56.906135,57.752356,NaN,NaN
4,Angola,AGO,"Life expectancy at birth, male (years)",SP.DYN.LE00.MA.IN,36.248000,34.266000,34.489000,34.720000,34.996000,35.236000,...,59.237000,59.727000,60.199000,60.609000,60.509000,60.498000,61.748000,62.099000,NaN,NaN


In [4]:
# check dimensions of data:

print("GDP shape:", gdp.shape)
print("Life Expectancy shape:", life.shape)

# filter out aggregated regions such as world, Europe & Central Asia, etc..

# Keep only rows with a 3-letter Country Code
gdp = gdp[gdp['Country Code'].str.len() == 3]
life = life[life['Country Code'].str.len() == 3]

print("Countries retained in GDP:", len(gdp))
print("Countries retained in Life Expectancy:", len(life))

# check for missing data

print("Missing GDP 2022 values:", gdp['2022'].isna().sum())
print("Missing Life Expectancy 2022 values:", life['2022'].isna().sum())

# preview column types

print(gdp.info())


GDP shape: (266, 70)
Life Expectancy shape: (266, 70)
Countries retained in GDP: 266
Countries retained in Life Expectancy: 266
Missing GDP 2022 values: 10
Missing Life Expectancy 2022 values: 1
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 266 entries, 0 to 265
Data columns (total 70 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Country Name    266 non-null    object 
 1   Country Code    266 non-null    object 
 2   Indicator Name  266 non-null    object 
 3   Indicator Code  266 non-null    object 
 4   1960            151 non-null    float64
 5   1961            154 non-null    float64
 6   1962            156 non-null    float64
 7   1963            156 non-null    float64
 8   1964            156 non-null    float64
 9   1965            162 non-null    float64
 10  1966            163 non-null    float64
 11  1967            167 non-null    float64
 12  1968            168 non-null    float64
 13  1969            168 

In [5]:
print(life.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 266 entries, 0 to 265
Data columns (total 70 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Country Name    266 non-null    object 
 1   Country Code    266 non-null    object 
 2   Indicator Name  266 non-null    object 
 3   Indicator Code  266 non-null    object 
 4   1960            263 non-null    float64
 5   1961            264 non-null    float64
 6   1962            264 non-null    float64
 7   1963            263 non-null    float64
 8   1964            263 non-null    float64
 9   1965            263 non-null    float64
 10  1966            264 non-null    float64
 11  1967            264 non-null    float64
 12  1968            264 non-null    float64
 13  1969            264 non-null    float64
 14  1970            264 non-null    float64
 15  1971            264 non-null    float64
 16  1972            264 non-null    float64
 17  1973            264 non-null    flo

### Step 4. Select relevant columns for 2022 and keep only countries with data in both datasets for that year

we will:

4.1. Subset both dataframes to 3 Columns: (Country Name, Country Code, 2022)
4.2. Drop rows with missing 2022 values
4.3. Merge on `Country Code` and `Country Name` to keep only intersecting countries
4.4. Inspect results

In [6]:
## 4.1 Subset both dataframes to 3 Columns: (Country Name, Country Code, 2022)

# Select relevant columns
gdp_2022 = gdp[['Country Name', 'Country Code', '2022']].copy()
life_2022 = life[['Country Name', 'Country Code', '2022']].copy()

# Rename for clarity
gdp_2022.rename(columns={'2022': 'GDP_PC_2022'}, inplace=True)
life_2022.rename(columns={'2022': 'LIFE_EX_YRS_2022'}, inplace=True)

In [7]:
## 4.2 Drop Missing 2022 Values

gdp_2022.dropna(subset=['GDP_PC_2022'], inplace=True)
life_2022.dropna(subset=['LIFE_EX_YRS_2022'], inplace=True)

print("Remaining countries with GDP data:", len(gdp_2022))
print("Remaining countries with Life Expectancy data:", len(life_2022))

Remaining countries with GDP data: 256
Remaining countries with Life Expectancy data: 265


In [8]:
## 4.3 Merge the Two Datasetes (inner join)

merged_2022 = pd.merge(
    gdp_2022,
    life_2022,
    on=['Country Name', 'Country Code'],
    how='inner'
)

print("Countries with data in both datasets:", len(merged_2022))

# Rank both columns (1 = highest value)
merged_2022['GDP_PC_RANK_2022'] = merged_2022['GDP_PC_2022'].rank(
    ascending=False, method='min'
)

merged_2022['LIFE_EX_YRS_RANK_2022'] = merged_2022['LIFE_EX_YRS_2022'].rank(
    ascending=False, method='min'
)

merged_2022.head(5)

Countries with data in both datasets: 256


,Country Name,Country Code,GDP_PC_2022,LIFE_EX_YRS_2022,GDP_PC_RANK_2022,LIFE_EX_YRS_RANK_2022
0,Aruba,ABW,30559.533535,73.537000,56.0,86.0
1,Africa Eastern and Southern,AFE,1628.318944,61.765707,216.0,220.0
2,Afghanistan,AFG,357.261153,63.941000,255.0,203.0
3,Africa Western and Central,AFW,1796.668633,56.906135,210.0,251.0
4,Angola,AGO,2929.694455,61.748000,187.0,221.0


In [9]:
## 4.4 Inspect reults
merged_2022.info()

print(merged_2022.head(5))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 256 entries, 0 to 255
Data columns (total 6 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Country Name           256 non-null    object 
 1   Country Code           256 non-null    object 
 2   GDP_PC_2022            256 non-null    float64
 3   LIFE_EX_YRS_2022       256 non-null    float64
 4   GDP_PC_RANK_2022       256 non-null    float64
 5   LIFE_EX_YRS_RANK_2022  256 non-null    float64
dtypes: float64(4), object(2)
memory usage: 12.1+ KB
                  Country Name Country Code   GDP_PC_2022  LIFE_EX_YRS_2022  \
0                        Aruba          ABW  30559.533535         73.537000   
1  Africa Eastern and Southern          AFE   1628.318944         61.765707   
2                  Afghanistan          AFG    357.261153         63.941000   
3   Africa Western and Central          AFW   1796.668633         56.906135   
4                       Angola  

### Step 5. Save the output data

ok, now that the data looks like its in the format we want, lets save it out as a .csv for import later

In [10]:
# Create the CLEAN directory if it doesn't exist
clean_dir.mkdir(parents=True, exist_ok=True)

output_path = clean_dir / "GDP_LifeExpectancy_2022_Clean.csv"

merged_2022.to_csv(output_path, index=False)

print(f"✅ Cleaned dataset saved successfully to:\n{output_path}")

✅ Cleaned dataset saved successfully to:
c:\Users\rl587\PROJECTS\NAULearningDataScience\Data\CLEAN\GDP_LifeExpectancy_2022_Clean.csv


In [11]:
# Check to make sure that all worked

check = pd.read_csv(output_path)
check.head()

,Country Name,Country Code,GDP_PC_2022,LIFE_EX_YRS_2022,GDP_PC_RANK_2022,LIFE_EX_YRS_RANK_2022
0,Aruba,ABW,30559.533535,73.537000,56.0,86.0
1,Africa Eastern and Southern,AFE,1628.318944,61.765707,216.0,220.0
2,Afghanistan,AFG,357.261153,63.941000,255.0,203.0
3,Africa Western and Central,AFW,1796.668633,56.906135,210.0,251.0
4,Angola,AGO,2929.694455,61.748000,187.0,221.0
